In [ ]:
!pip install openpyxl requests fpdf pillow


In [ ]:
import pandas as pd
import requests
from fpdf import FPDF
from PIL import Image
from google.colab import files
import os

# Carica file Excel e immagini manualmente
print("Carica ora il file Excel con i dati dei prodotti:")
uploaded_excel = files.upload()

print("Ora carica una ad una le immagini dei prodotti:")
uploaded_images = files.upload()

# Recupera nome file Excel
excel_filename = list(uploaded_excel.keys())[0]
df = pd.read_excel(excel_filename)

# Imposta la tua API key
OPENROUTER_API_KEY = input("Inserisci la tua API Key OpenRouter: ")


In [ ]:
def safe_text(text):
    replacements = {
        "’": "'",
        "‘": "'",
        "“": '"',
        "”": '"',
        "–": "-",
        "—": "-",
        "…": "...",
        "€": "EUR",
        "•": "-",
    }
    for wrong, right in replacements.items():
        text = text.replace(wrong, right)
    return text.encode("latin-1", "ignore").decode("latin-1")


In [ ]:
def fetch_gpt_output(name, category, materials, target, tone, price):
    prompt = f"""Crea una scheda prodotto composta da:
1. Descrizione e-commerce (max 80 parole). Includi il prezzo (EUR {price}) alla fine.
2. Caption per Instagram/Facebook (max 200 caratteri, tono umano, con prezzo visibile).
3. Sintesi breve da usare in catalogo (1 frase con prezzo).

Usa queste informazioni:
- Nome prodotto: {name}
- Categoria: {category}
- Materiali: {materials}
- Target: {target}
- Tono: {tone}
- Prezzo: EUR {price}
"""
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "openai/gpt-4-turbo",
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    content = response.json()["choices"][0]["message"]["content"]
    return content


In [ ]:
def create_pdf(name, content, image_path):
    pdf = FPDF()
    pdf.add_page()

    if image_path and os.path.exists(image_path):
        try:
            img = Image.open(image_path)
            img.thumbnail((300, 300))
            temp_path = "/tmp/preview.jpg"
            img.save(temp_path)
            pdf.image(temp_path, x=10, y=10, w=90)
            pdf.set_y(70)
        except Exception as e:
            pdf.set_y(20)
            pdf.set_font("Arial", 'I', 10)
            pdf.cell(0, 10, f"(Immagine non inserita: {e})", ln=True)

    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, f"Product Sheet: {safe_text(name)}", ln=True)

    pdf.set_font("Arial", '', 12)
    pdf.ln(10)
    pdf.multi_cell(0, 10, safe_text(content))

    file_name = f"{name.replace(' ', '_')}.pdf"
    file_path = f"/content/{file_name}"
    pdf.output(file_path)
    return file_path


In [ ]:
print("🚀 Generazione schede in corso...")

pdf_paths = []

for index, row in df.iterrows():
    name = str(row['nome'])
    category = str(row['categoria'])
    materials = str(row['materiali'])
    target = str(row['target'])
    tone = str(row['tono'])
    price = str(row['prezzo'])
    image_file = str(row['immagine']).strip()

    gpt_output = fetch_gpt_output(name, category, materials, target, tone, price)
    print(f"Output GPT per '{name}'")

    pdf_path = create_pdf(name, gpt_output, f"/content/{image_file}")
    pdf_paths.append(pdf_path)
    print(f"PDF salvato: {pdf_path}")
